In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [ ]:
import os
import glob
import numpy as np
import rasterio
from rasterio.merge import merge
from scipy.stats import linregress

## Directory where rasters are stored
#Updated: 10/14/25
raster_dir = "F:/Biomass tiffs/Biomass modeling/GA and VA/Model output LTERs/ABG_firstrun/GEE_Exports"

##Output directories

output_dir = r"F:/Biomass tiffs/Biomass modeling/GA and VA/All_trend"
os.makedirs(output_dir, exist_ok=True)

# --- Step 1: Group all tiles by year ---
all_files = glob.glob(os.path.join(raster_dir, "*.tif"))
year_groups = {}

for f in all_files:
    base = os.path.basename(f)
    # Extract the year: assumes pattern like "ABG_1984-0000000000-0000000000.tif"
    year_str = base.split("_")[1].split("-")[0]
    try:
        year = int(year_str)
        year_groups.setdefault(year, []).append(f)
    except ValueError:
        continue

# --- Step 2: Mosaic tiles for each year ---
mosaicked_files = []
for year, files in sorted(year_groups.items()):
    if not files:
        continue
    srcs = [rasterio.open(f) for f in files]
    
    # Merge (mosaic) the tiles
    mosaic, out_trans = merge(srcs)
    profile = srcs[0].profile
    profile.update({
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "float32",
        "count": 1
    })
    
    # Optional: skip mosaics with no valid data
    if np.all(np.isnan(mosaic)):
        print(f"Skipping {year}: empty mosaic.")
        for s in srcs:
            s.close()
        continue

    out_path = os.path.join(output_dir, f"ABG_{year}_mosaic.tif")
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(mosaic[0], 1)
    for s in srcs:
        s.close()
    mosaicked_files.append(out_path)

print(f"Mosaicked {len(mosaicked_files)} years of data.")

# --- Step 3: Stack all yearly mosaics ---
raster_files = sorted(mosaicked_files)
years = np.array([int(os.path.basename(f).split('_')[1]) for f in raster_files])

stack = []
for file in raster_files:
    with rasterio.open(file) as src:
        arr = src.read(1).astype(np.float32)
        nodata = src.nodata
        arr[arr == nodata] = np.nan
        stack.append(arr)
        profile = src.profile  # keep one profile for writing later

stack = np.stack(stack)  # (time, rows, cols)

# --- Step 4: Compute slope and significance per pixel ---
rows, cols = stack.shape[1:]
slope_array = np.full((rows, cols), np.nan, dtype=np.float32)
pval_array = np.full((rows, cols), np.nan, dtype=np.float32)

for i in range(rows):
    for j in range(cols):
        y = stack[:, i, j]
        if np.isfinite(y).sum() > 1:
            res = linregress(years, y)
            slope_array[i, j] = res.slope
            pval_array[i, j] = res.pvalue

# --- Step 5: Save slope and significance rasters ---
slope_out = os.path.join(output_dir, "LOG_1984to2023.tif")
profile.update(dtype=rasterio.float32, count=1, nodata=np.nan)

with rasterio.open(slope_out, "w", **profile) as dst:
    dst.write(slope_array, 1)

sig_mask = (pval_array < 0.05)
sig_slopes = np.where(sig_mask, slope_array, np.nan)
sig_out = os.path.join(output_dir, "LOG_sig_1984to2023.tif")

with rasterio.open(sig_out, "w", **profile) as dst:
    dst.write(sig_slopes, 1)

print(f"Slope map saved: {slope_out}")
print(f"Significant slope map saved: {sig_out}")


In [ ]:
import os
import glob
import numpy as np
import rasterio
from scipy.stats import linregress

rows, cols = stack.shape[1:]
slope_array = np.full((rows, cols), np.nan, dtype=np.float32)
pval_array = np.full((rows, cols), np.nan, dtype=np.float32)

# Calculate slope + p-value per pixel
for i in range(rows):
    for j in range(cols):
        y = stack[:, i, j]
        if np.isfinite(y).sum() > 1:  # need at least 2 points
            res = linregress(years, y)
            slope_array[i, j] = res.slope
            pval_array[i, j] = res.pvalue

# Save slope raster
slope_out = "F:/Biomass tiffs/Biomass modeling/Georgia/All_trend/LOG_1984to2023.tif"
profile.update(dtype=rasterio.float32, count=1, nodata=np.nan)
with rasterio.open(slope_out, 'w', **profile) as dst:
    dst.write(slope_array, 1)

# Save significant slopes only (p < 0.05)
sig_mask = (pval_array < 0.05)
sig_slopes = np.where(sig_mask, slope_array, np.nan)
sig_out = "F:/Biomass tiffs/Biomass modeling/Georgia/All_trend/LOG_sig_1984to2023.tif"
with rasterio.open(sig_out, 'w', **profile) as dst:
    dst.write(sig_slopes, 1)

print(f"Slope map saved: {slope_out}")
print(f"Significant slope map saved: {sig_out}")